This file exports the links and nodes to qgis, such that we can perform some processing on it to obtain the links inside a zone or cordon to toll. 

In [ ]:
import os
import geopandas as gpd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore') # hide warnings
sys.path.append("..")
from pyproj import Proj, transform
from pickle import load 
from shapely.geometry import Point, LineString

Change the data paths and the crs for the qgis layer according to your needs. 

In [ ]:
layer_crs = 'epsg:31370'

data_path = os.path.dirname(os.path.realpath("__file__")) + os.path.sep + 'data_map'
# Input = network: 
HEEDS_input_path = data_path + os.path.sep + "HEEDS_input" + os.path.sep
network_path = HEEDS_input_path + "network_with_centroids" + os.path.sep + "elastic_BRUSSEL_40"
x_centroids_path = HEEDS_input_path + "elastic"
y_centroids_path = HEEDS_input_path + "elastic" 
# Output:
node_shapefile_path =  data_path + os.path.sep + 'QGIS' + os.path.sep + "nodes_BRUSSEL_40.shp"
link_shapefile_path =  data_path + os.path.sep + 'QGIS' + os.path.sep + "links_BRUSSEL_40.shp"

4. What we need to export to QGIS:

- All intersection nodes, with attributes: x coordinate, y coordinate and node_id. 
- All non connector links, with attributes: link_id, from_node_id and to_node_id. 


In [ ]:
# Load required files 
with open(network_path, 'rb') as network_file:
    network = load(network_file)
x_centroids = np.loadtxt(x_centroids_path)
y_centroids = np.loadtxt(y_centroids_path)

# Remove centroids 
nr_of_centroids = len(x_centroids)
intersection_nodes = list(network.nodes.data())[nr_of_centroids:]

# Remove connectors
edge_data = [(_, _, data) for _, _, data in network.edges.data()]
sorted_edges = sorted(edge_data, key=lambda t: t[2]["link_id"])
link_types = np.array([np.int8(d.get("link_type", 0)) for (_, _, d) in sorted_edges], dtype=np.int8)
connector_ids = []
for i in range(len(link_types)):
    if link_types[i] == -1 or link_types[i] == 1:
        connector_ids.append(i)
for i in reversed(connector_ids):
    del sorted_edges[i]

In [ ]:
node_ids = []
x_coord = []
y_coord = []
for _, data in intersection_nodes:
    node_ids.append(data['node_id'])
    x_coord.append(data['x_coord'])
    y_coord.append(data['y_coord'])
x_coord, y_coord = transform(Proj(init='epsg:4326'),Proj(init=layer_crs),x_coord,y_coord)

link_ids = []
from_node_ids = []
to_node_ids = []
for _, _, data in sorted_edges:
    link_ids.append(data['link_id'])
    from_node_ids.append(data['from_node_id'])
    to_node_ids.append(data['to_node_id'])

In [ ]:
# Create Point shapefile for the nodes 
nodes = []

# Iterate over the node lists and add each node to the dictionary
for node_id, x, y in zip(node_ids, x_coord, y_coord):
    nodes.append({'node_id': node_id, 'x': x, 'y': y})

points = [Point(node['x'], node['y']) for node in nodes]

nodes_shapefile = gpd.GeoDataFrame(nodes, geometry=points)
nodes_shapefile.crs = {'init': layer_crs}
nodes_shapefile.to_file(node_shapefile_path)

In [ ]:
# Create Line shapefile for the links 
links = []
lines = []

for link_id, from_id, to_id in zip(link_ids, from_node_ids, to_node_ids):
    links.append({'link_id': link_id, 'from_node_id': from_id, 'to_node_id': to_id})
    from_node = [node for node in nodes if node['node_id'] == from_id][0]
    to_node = [node for node in nodes if node['node_id'] == to_id][0]
    line = LineString([(from_node['x'], from_node['y']), (to_node['x'], to_node['y'])])
    lines.append(line)

links_shapefile = gpd.GeoDataFrame(links, geometry=lines)
links_shapefile.crs = {'init': layer_crs}
links_shapefile.to_file(link_shapefile_path)

    Congratulations, you have been able to export everything you need to QGIS, needed to find accurate cordon- or zone-based tolling schemes!
There you can create a desired polygon and use tools to select links that cross the polygon (cordon) or links that lie within the polygon (zone)